In [38]:
import pandas as pd
import dask.dataframe as dd
import dask.bag as db
from gensim.models import Word2Vec
docs = dd.read_parquet("output.pq/")

In [39]:
texts = docs['News_Tokens'].compute()
bag = db.from_sequence(texts)
sentences: list[str] = bag.flatten().map(lambda x: x.split()).compute()

In [40]:
print(len(sentences))

423175


In [33]:
model = Word2Vec(sentences=sentences, vector_size=100, window=5,min_count=1, sg=0)
model.save('word2vec/word2vec_sent_5_sgu.model')

# Построение тезауруса

In [7]:
import pandas as pd
import dask.dataframe as dd
import dask.bag as db
from gensim.models import Word2Vec
docs = dd.read_parquet("output.pq/")

In [34]:
model = Word2Vec.load("word2vec/word2vec_sent_5_sgu.model")
texts = docs['News_Tokens'].compute()
bag = db.from_sequence(texts)
sentences = bag.flatten()

In [35]:
words = sentences.map(lambda x: x.split()).flatten().distinct().compute()


In [19]:
print(len(words))

45204


In [ ]:
results = []
threshold = 0.7
for word in words:
    try:
        # Получаем 10 ближайших слов
        similar_words = model.wv.most_similar(word, topn=10)
        filtered_words = [(w, sim) for w, sim in similar_words if sim >= threshold]
        results.append([word, filtered_words])
    except KeyError:
        # Если слово не в модели, пропускаем его
        print(f"Слово '{word}' не найдено в модели.")

df_results = pd.DataFrame(results, columns=["Word", "Most_Similar_Word"])

df_results.to_csv("word2vec/similar_words_sent_5.csv", index=False)